**3장 – 분류**

_이 노트북은 3장의 모든 샘플 코드와 연습 문제 정답을 담고 있습니다._

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/rickiepark/handson-ml3/blob/main/03_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# 3.0 설정

파이썬 3.7 또는 그 이상이 필요합니다:

In [1]:
import sys

assert sys.version_info >= (3, 7)

Scikit-Learn ≥1.0.1이 필요합니다:

In [2]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

이전 장과 마찬가지로 그래프를 보기 좋게 그리기 위해 기본 폰트 크기를 설정합니다:

In [3]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

`images/classification` 폴더가 없다면 이 폴더를 만들고 고해상도 이미지 저장을 위해 노트북에서 사용할 `save_fig()` 함수를 정의합니다:

In [4]:
from pathlib import Path

IMAGES_PATH = Path() / "images"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 연습문제 해답

## 1. 97% 정확도의 MNIST 분류기

문제: _MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요. 힌트: `KNeighborsClassifier`가 이 작업에 아주 잘 맞습니다. 좋은 하이퍼파라미터 값만 찾으면 됩니다(`weights`와 `n_neighbors` 하이퍼파라미터로 그리드 탐색을 시도해보세요)._

K-최근접 이웃 분류기를 사용해 테스트 세트에서 성능을 측정해 보죠. 이 모델을 기준 모델로 삼습니다:

In [ ]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)
baseline_accuracy = knn_clf.score(X_test, y_test)
baseline_accuracy

훌륭합니다! 기본 하이퍼파라미터를 사용하는 KNN 분류기는 우리 목표에 이미 매우 근접합니다.

하이퍼파라미터를 튜닝하면 도움이 될 수 있는지 살펴봅시다. 탐색 속도를 높이기 위해 처음 10,000개의 이미지에 대해서만 훈련해 보겠습니다:

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5, 6]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5)
grid_search.fit(X_train[:10_000], y_train[:10_000])

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

점수가 떨어졌지만 10,000개의 이미지로만 훈련했기 때문에 예상했던 결과입니다. 이제 최상의 모델을 가져와 전체 훈련 세트에서 다시 훈련해 보겠습니다:

In [ ]:
grid_search.best_estimator_.fit(X_train, y_train)
tuned_accuracy = grid_search.score(X_test, y_test)
tuned_accuracy

97% 정확도 목표에 도달했습니다! 🥳

## 2. 데이터 증식

문제: _MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어보세요. `scipy.ndimage.interpolation` 모듈의 `shift()` 함수를 사용할 수 있습니다. 예를 들어 `shift(image, [2, 1], cval=0)`은 아래로 2픽셀, 오른쪽으로 1픽셀 이동시킵니다. 그런 다음 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요. 마지막으로 이 확장된 데이터셋에서 앞에서 찾은 최선의 모델을 훈련시키고 테스트 세트에서 정확도를 측정해보세요. 모델 성능이 더 높아졌는지 확인해보세요! 인위적으로 훈련 세트를 늘리는 이 기법을 _데이터 증식_ 또는 _훈련 세트 확장_(training set expansion)이라고 합니다._

각 이미지의 약간 변형된 버전을 추가하여 MNIST 데이터셋을 증식해 보겠습니다.

In [95]:
from scipy.ndimage import shift

In [96]:
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

작동하는지 확인해 보겠습니다:

In [ ]:
image = X_train[1000]  # 데모에 사용할 임의의 숫자
shifted_image_down = shift_image(image, 0, 5)
shifted_image_left = shift_image(image, -5, 0)

plt.figure(figsize=(12, 3))
plt.subplot(131)
plt.title("Original")
plt.imshow(image.reshape(28, 28),
           interpolation="nearest", cmap="Greys")
plt.subplot(132)
plt.title("Shifted down")
plt.imshow(shifted_image_down.reshape(28, 28),
           interpolation="nearest", cmap="Greys")
plt.subplot(133)
plt.title("Shifted left")
plt.imshow(shifted_image_left.reshape(28, 28),
           interpolation="nearest", cmap="Greys")
plt.show()

좋아 보이네요! 이제 모든 이미지를 왼쪽, 오른쪽, 위, 아래로 1픽셀씩 이동하여 증식된 훈련 세트를 만들어 보겠습니다:

In [98]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((-1, 0), (1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

증식된 훈련 세트를 섞어 보겠습니다. 그렇지 않으면 이동된 모든 이미지가 함께 묶입니다:

In [99]:
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

이제 이전 연습문제에서 찾은 최상의 하이퍼파라미터를 사용하여 모델을 훈련해 보겠습니다:

In [100]:
knn_clf = KNeighborsClassifier(**grid_search.best_params_)

In [ ]:
knn_clf.fit(X_train_augmented, y_train_augmented)

**경고**: 다음 셀을 실행하는 데 몇 분 정도 걸릴 수 있습니다:

In [ ]:
augmented_accuracy = knn_clf.score(X_test, y_test)
augmented_accuracy

단순히 데이터를 보강하는 것만으로도 정확도가 0.5% 향상되었습니다. 그다지 인상적으로 들리지 않을 수도 있지만 실제로는 오류율이 크게 감소했음을 의미합니다:

In [ ]:
error_rate_change = (1 - augmented_accuracy) / (1 - tuned_accuracy) - 1
print(f"error_rate_change = {error_rate_change:.0%}")

데이터 증식 덕분에 오류율이 상당히 감소했습니다.

## 3. 타이타닉 데이터셋 도전하기

문제: _타이타닉(Titanic) 데이터셋에 도전해보세요. [캐글](https://www.kaggle.com/c/titanic)이 시작하기 좋습니다. 또는 https://homl.info/titanic.tgz 에서 데이터를 다운로드하고, 2장의 주택 데이터에서 했던 것처럼 이 파일의 압축을 풀 수 있습니다. 이렇게 하면 CSV 파일 train.csv와 test.csv가 두 개가 생성되며, 이 파일을 `pandas.read_csv()`를 사용해 로드할 수 있습니다. 다른 열을 기반으로 `Survived` 열을 예측할 수 있는 분류기를 훈련하는 것이 목표입니다._

데이터를 가져와서 로드해 보겠습니다:

In [104]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_titanic_data():
    tarball_path = Path("datasets/titanic.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/titanic.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as titanic_tarball:
            titanic_tarball.extractall(path="datasets")
    return [pd.read_csv(Path("datasets/titanic") / filename)
            for filename in ("train.csv", "test.csv")]

In [105]:
train_data, test_data = load_titanic_data()

이 데이터는 이미 훈련 세트와 테스트 세트로 분할되어 있습니다. 하지만 테스트 데이터에는 레이블이 포함되어 있지 않습니다. 목표는 훈련 데이터에서 최상의 모델을 훈련한 다음, 테스트 데이터에서 예측을 만들고 캐글에 업로드하여 최종 점수를 확인하는 것입니다.

훈련 세트의 상위 몇 줄을 살펴봅시다:

In [ ]:
train_data.head()

특성은 다음과 같은 의미를 갖습니다:
* **PassengerId**: 각 승객의 고유 식별자
* **Survived**: 타깃입니다. 0은 승객이 생존하지 못했음을 의미하고, 1은 생존했음을 의미합니다.
* **Pclass**: 객실 등급.
* **Name**, **Sex**, **Age**: 설명이 필요 없는 특성
* **SibSp**: 타이타닉 호에 탑승한 승객의 형제자매 및 배우자 수.
* **Parch**: 타이타닉에 탑승한 승객의 자녀 및 부모 수.
* **Ticket**: 티켓 ID
* **Fare**: 지불한 가격(파운드)
* **Cabin**: 승객의 객실 번호
* **Embarked**: 승객이 타이타닉 호에 승선한 장소

승객의 나이, 성별, 객실 등급, 탑승 장소 등의 특성을 기반으로 승객의 생존 여부를 예측하는 것이 목표입니다.

명시적으로 `PassengerId` 열을 인덱스 열로 설정해 보겠습니다:

In [107]:
train_data = train_data.set_index("PassengerId")
test_data = test_data.set_index("PassengerId")

누락된 데이터의 양을 확인하기 위해 자세한 정보를 확인해 보겠습니다:

In [ ]:
train_data.info()

In [ ]:
train_data[train_data["Sex"]=="female"]["Age"].median()

**Age**, **Cabin** 및 **Embarked** 특성은 (891개 non-null이 아니기 때문에) 때때로 null입니다. 특히 **Cabin**(77%가 null)이 그렇습니다. 지금은 **Cabin**은 무시하고 나머지에 집중하겠습니다. **Age** 특성에는 약 19%의 null 값이 있으므로 이를 어떻게 처리할지 결정해야 합니다. null 값을 평균 연령으로 대체하는 것이 합리적입니다. 다른 열을 기반으로 나이를 예측하면 좀 더 스마트할 수 있지만(예를 들어, 1등급 객실의 나이 중앙값은 37세, 2등급은 29세, 3등급은 24세), 여기서는 간단하게 전체 중앙값을 사용하기로 하겠습니다.

**Name** 및 **Ticket** 특성은 어느 정도 가치가 있을 수 있지만 모델이 사용할 수 있는 유용한 숫자로 변환하기에는 약간 까다롭습니다. 따라서 지금은 무시하겠습니다.

숫자 특성을 살펴보겠습니다:

In [ ]:
train_data.describe()

* 이런, **Survived**가 38%뿐입니다! 😭 40%에 충분히 근접한 수치이므로 정확도는 모델을 평가하는 합리적인 지표가 될 것입니다.
* 평균 **Fare**는 32.20파운드로 그렇게 비싸지 않은 것 같습니다(하지만 그 당시에는 많은 돈이었을 것입니다).
* 평균 **Age**는 30세 미만이었습니다.

타깃이 실제로 0 또는 1인지 확인해 보겠습니다:

In [ ]:
train_data["Survived"].value_counts()

이제 모든 범주형 특성에 대해 간단히 살펴보겠습니다:

In [ ]:
train_data["Pclass"].value_counts()

In [ ]:
train_data["Sex"].value_counts()

In [ ]:
train_data["Embarked"].value_counts()

승선 위치 특성은 승객이 승선한 위치를 알려줍니다: C=Cherbourg, Q=Queenstown, S=Southampton.

이제 숫자 특성에 대한 파이프라인부터 시작하여 전처리 파이프라인을 구축해 보겠습니다:

In [115]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

이제 범주형 특성에 대한 파이프라인을 구축할 수 있습니다:

In [116]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [117]:
# 사이킷런 1.2버전에서 OneHotEncoder의 `sparse_output` 매개변수가 추가되었고
# `sparse` 매개변수는 1.4버전에서 삭제되었습니다.
cat_pipeline = Pipeline([
        ("ordinal_encoder", OrdinalEncoder()),
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse_output=False)),
    ])

마지막으로 숫자 파이프라인과 범주 파이프라인을 결합해 보겠습니다:

In [118]:
from sklearn.compose import ColumnTransformer

num_attribs = ["Age", "SibSp", "Parch", "Fare"]
cat_attribs = ["Pclass", "Sex", "Embarked"]

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

멋지네요! 이제 원시 데이터를 가져와 원하는 머신 러닝 모델에 공급 가능한 수치 입력 특성을 출력하는 멋진 전처리 파이프라인이 생겼습니다.

In [ ]:
X_train = preprocess_pipeline.fit_transform(train_data)
X_train

레이블을 챙기는 것도 잊지 마세요:

In [120]:
y_train = train_data["Survived"]

이제 분류기를 훈련할 준비가 되었습니다. 먼저 `RandomForestClassifier`로 시작해 보죠:

In [ ]:
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(X_train, y_train)

모델이 학습되었으니 이제 이 모델을 사용하여 테스트 세트에 대한 예측을 수행해 보겠습니다:

In [122]:
X_test = preprocess_pipeline.transform(test_data)
y_pred = forest_clf.predict(X_test)

이제 이러한 예측이 포함된 CSV 파일을 (캐글이 기대하는 형식에 맞추어) 작성한 다음 업로드하고 최선이기를 바랄 수 있습니다. 하지만 잠깐만요! 그냥 희망하는 것보다 더 좋은 방법이 있습니다. 교차 검증을 사용하여 모델이 얼마나 좋은지 알아보는 것은 어떨까요?

In [ ]:
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

좋아요, 나쁘지 않네요! 캐글에서 타이타닉 대회의 [리더보드](https://www.kaggle.com/c/titanic/leaderboard)를 보면, 이 점수가 상위 2%에 속하는 것을 확인할 수 있습니다, 우후! 100% 정확도를 달성한 캐글러도 있었지만, 타이타닉의 [희생자 명단](https://www.encyclopedia-titanica.org/titanic-victims/)을 쉽게 찾아볼 수 있는 것으로 보아 머신러닝의 영향이 거의 없었던 것 같습니다! 😆

`SVC`를 시도해 보죠:

In [ ]:
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto")
svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
svm_scores.mean()

좋아요! 이 모델이 더 좋아 보입니다.

하지만 10개의 교차 검증 폴드에 대한 평균 정확도만 보는 대신 각 모델에 대한 10개의 모든 점수를 하위 및 상위 사분위수를 강조하는 박스 플롯과 점수의 범위를 보여주는 '수염'을 함께 그려 보겠습니다(이 시각화를 제안해 준 Nevin Yilmaz에게 감사드립니다). `boxplot()` 함수는 이상치을 감지하며 수염에는 포함하지 않는다는 점에 유의하세요. 구체적으로, 하위 사분위수가 $Q_1$이고 상위 사분위수가 $Q_3$인 경우 사분위수 간 범위가 $IQR = Q_3 - Q_1$(상자의 높이)이며, $Q_1 - 1.5 \times IQR$보다 낮은 점수는 이상치이고, $Q3 + 1.5 \times IQR$보다 큰 점수도 이상치에 해당합니다.

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot([1]*10, svm_scores, ".")
plt.plot([2]*10, forest_scores, ".")
plt.boxplot([svm_scores, forest_scores], labels=("SVM", "Random Forest"))
plt.ylabel("Accuracy")
plt.show()

랜덤 포레스트 분류기는 10개 폴드 중 하나에서 매우 높은 점수를 받았지만 전반적으로 평균 점수가 낮고 더 널리 퍼져 있어서 SVM 분류기가 더 잘 일반화할 가능성이 높은 것으로 보입니다.

이 결과를 더욱 개선하려면 다음과 같이 할 수 있습니다:
* 교차 검증 및 그리드 검색을 사용하여 더 많은 모델을 비교하고 하이퍼파라미터를 조정하세요.
* 더 많은 특성 공학을 수행하세요. 예를 들면 다음과 같습니다:
  * 숫자 특성을 범주 특성으로 변환해 보세요. 예를 들어, 연령대별로 생존율이 매우 다르므로(아래 참조) 연령 버킷 카테고리를 만들어 연령 대신 사용하는 것이 도움이 될 수 있습니다. 마찬가지로, 혼자 여행하는 사람들의 생존율이 30%에 불과했기 때문에 혼자 여행하는 사람들을 위한 특별 카테고리를 만드는 것이 유용할 수 있습니다(아래 참조).
  * **SibSp**와 **Parch**를 합쳐 보세요.
  * **Survived** 속성과 잘 연관되는 이름 부분을 찾아보세요.
  * **Cabin** 열을 사용하세요. 예를 들어 첫 글자를 가져와 범주형 속성으로 처리합니다.

In [ ]:
train_data["AgeBucket"] = train_data["Age"] // 15 * 15
train_data[["AgeBucket", "Survived"]].groupby(['AgeBucket']).mean()

In [ ]:
train_data["RelativesOnboard"] = train_data["SibSp"] + train_data["Parch"]
train_data[["RelativesOnboard", "Survived"]].groupby(
    ['RelativesOnboard']).mean()

## 4. 스팸 필터

문제: _스팸 분류기를 만들어보세요(아주 도전적인 과제입니다)._

* _[아파치 스팸어새신(Apache SpamAssassin) 공공 데이터셋](https://homl.info/spamassassin)에서 스팸(spam)과 햄(ham)(스팸이 아닌 메일) 샘플을 내려받습니다._
* _데이터셋의 압축을 풀고 데이터 형식을 살펴봅니다._
* _데이터셋을 훈련 세트와 테스트 세트로 나눕니다._
* _각 이메일을 특성 벡터로 변환하는 데이터 준비 파이프라인을 만듭니다. 이 준비 파이프라인은 하나의 이메일을 가능한 단어의 존재 여부를 나타내는 (희소) 벡터로 바꿔야 합니다. 예를 들어 모든 이메일이 네 개의 단어 ‘Hello’, ‘how’, ‘are’, ‘you’만 포함한다면 ‘Hello you Hello Hello you’란 이메일은 벡터 [1, 0, 0, 1]([‘Hello’ 있음, ‘how’ 없음, ‘are’ 없음, ‘you’ 있음]을 의미)로 변환되거나, 단어의 출현 횟수에 관심이 있다면 [3, 0, 0, 2]로 변환되어야 합니다._


_준비 파이프라인에 이메일 헤더 제거, 소문자 변환, 구두점 제거, 모든 URLs 주소를 ‘URL’로 대체, 모든 숫자를 ‘NUMBER’로 대체, 어간stem 추출20(즉, 단어의 끝을 떼어냅니다. 이런 작업을 할 수 있는 파이썬 라이브러리가 있습니다) 등을 수행할지 여부를 제어하기 위해 하이퍼파라미터를 추가합니다._

_마지막으로 여러 분류기를 시도해보고 재현율과 정밀도가 모두 높은 스팸 분류기를 만들 수 있는지 확인해보세요._

In [128]:
import tarfile

def fetch_spam_data():
    spam_root = "http://spamassassin.apache.org/old/publiccorpus/"
    ham_url = spam_root + "20030228_easy_ham.tar.bz2"
    spam_url = spam_root + "20030228_spam.tar.bz2"

    spam_path = Path() / "datasets" / "spam"
    spam_path.mkdir(parents=True, exist_ok=True)
    for dir_name, tar_name, url in (("easy_ham", "ham", ham_url),
                                    ("spam", "spam", spam_url)):
        if not (spam_path / dir_name).is_dir():
            path = (spam_path / tar_name).with_suffix(".tar.bz2")
            print("Downloading", path)
            urllib.request.urlretrieve(url, path)
            tar_bz2_file = tarfile.open(path)
            tar_bz2_file.extractall(path=spam_path)
            tar_bz2_file.close()
    return [spam_path / dir_name for dir_name in ("easy_ham", "spam")]

In [ ]:
ham_dir, spam_dir = fetch_spam_data()

다음, 모든 이메일을 읽어 들입니다:

In [130]:
ham_filenames = [f for f in sorted(ham_dir.iterdir()) if len(f.name) > 20]
spam_filenames = [f for f in sorted(spam_dir.iterdir()) if len(f.name) > 20]

In [ ]:
len(ham_filenames)

In [ ]:
len(spam_filenames)

파이썬의 `email` 모듈을 사용해 이메일을 파싱합니다(헤더, 인코딩 등을 처리합니다):

In [133]:
import email
import email.policy

def load_email(filepath):
    with open(filepath, "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [134]:
ham_emails = [load_email(filepath) for filepath in ham_filenames]
spam_emails = [load_email(filepath) for filepath in spam_filenames]

데이터가 어떻게 구성되어 있는지 감을 잡기 위해 햄 메일과 스팸 메일을 하나씩 보겠습니다:

In [ ]:
print(ham_emails[1].get_content().strip())

In [ ]:
print(spam_emails[6].get_content().strip())

어떤 이메일은 이미지나 첨부 파일을 가진 멀티파트(multipart)입니다(메일에 포함되어 있을수 있습니다). 어떤 파일들이 있는지 살펴 보겠습니다:

In [137]:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        multipart = ", ".join([get_email_structure(sub_email)
                               for sub_email in payload])
        return f"multipart({multipart})"
    else:
        return email.get_content_type()

In [138]:
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [ ]:
structures_counter(ham_emails).most_common()

In [ ]:
structures_counter(spam_emails).most_common()

햄 메일은 평범한 텍스트가 많고 스팸은 HTML일 경우가 많습니다. 적은 수의 햄 이메일이 PGP로 서명되어 있지만 스팸 메일에는 없습니다. 요약하면 이메일 구조는 유용한 정보입니다.

이제 이메일 헤더를 살펴보겠습니다:

In [ ]:
for header, value in spam_emails[0].items():
    print(header, ":", value)

보낸사람의 이메일 주소와 같이 헤더에는 유용한 정보가 많이 있지만 여기서는 `Subject` 헤더만 다뤄 보겠습니다:

In [ ]:
spam_emails[0]["Subject"]

좋습니다. 데이터에를 더 살펴보기 전에 훈련 세트와 테스트 세트로 나누도록 하겠습니다:

In [143]:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(ham_emails + spam_emails, dtype=object)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

이제 전처리 함수를 작성하겠습니다. 먼저 HTML을 일반 텍스트로 변환하는 함수가 필요합니다. 이 작업에는 당연히 [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) 라이브러리를 사용하는게 좋지만 의존성을 줄이기 위해서 정규식을 사용하여 대강 만들어 보겠습니다([un̨ho͞ly radiańcé destro҉ying all enli̍̈́̂̈́ghtenment](https://stackoverflow.com/a/1732454/38626)의 위험에도 불구하고). 다음 함수는 `<head>` 섹션을 삭제하고 모든 `<a>` 태그를 HYPERLINK 문자로 바꿉니다. 그런 다음 모든 HTML 태그를 제거하고 텍스트만 남깁니다. 보기 편하게 여러개의 개행 문자를 하나로 만들고 (`&gt;`나 `&nbsp;` 같은) html 엔티티를 복원합니다:

In [144]:
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)

잘 작동하는지 확인해 보겠습니다. 다음은 HTML 스팸입니다:

In [ ]:
html_spam_emails = [email for email in X_train[y_train==1]
                    if get_email_structure(email) == "text/html"]
sample_html_spam = html_spam_emails[7]
print(sample_html_spam.get_content().strip()[:1000], "...")

변환된 텍스트입니다:

In [ ]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000], "...")

아주 좋습니다! 이제 포맷에 상관없이 이메일을 입력으로 받아서 일반 텍스트를 출력하는 함수를 만들겠습니다:

In [147]:
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)

In [ ]:
print(email_to_text(sample_html_spam)[:100], "...")

자연어 처리 툴킷([NLTK](http://www.nltk.org/))을 사용해 어간 추출을 해보죠:

In [ ]:
import nltk

stemmer = nltk.PorterStemmer()
for word in ("Computations", "Computation", "Computing", "Computed", "Compute",
             "Compulsive"):
    print(word, "=>", stemmer.stem(word))

인터넷 주소는 "URL" 문자로 바꾸겠습니다. [정규식](https://mathiasbynens.be/demo/url-regex)을 하드 코딩할 수도 있지만 [urlextract](https://github.com/lipoja/URLExtract) 라이브러리를 사용하겠습니다:

In [150]:
# 코랩이나 캐글을 사용하나요?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# 코랩이나 캐글에서 이 노트북을 실행하려면 먼저 pip install urlextract을 실행합니다
if IS_COLAB or IS_KAGGLE:
    %pip install -q -U urlextract

**노트:** 주피터 노트북에서는 항상 `!pip` 대신 `%pip`를 사용해야 합니다. `!pip`는 다른 환경에 라이브러리를 설치할 수 있기 때문입니다. 반면 `%pip`는 현재 실행 중인 환경에 설치됩니다.

In [ ]:
import urlextract # 루트 도메인 이름을 다운로드하기 위해 인터넷 연결이 필요할지 모릅니다

url_extractor = urlextract.URLExtract()
some_text = "Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s"
print(url_extractor.find_urls(some_text))

이들을 모두 하나의 변환기로 연결하여 이메일을 단어 카운트로 바꿀 것입니다. 파이썬의 `split()` 메서드를 사용하면 구둣점과 단어 경계를 기준으로 문장을 단어로 바꿉니다. 이 방법이 많은 언어에 통하지만 전부는 아닙니다. 예를 들어 중국어와 일본어는 일반적으로 단어 사이에 공백을 두지 않습니다. 베트남어는 음절 사이에 공백을 두기도 합니다. 여기서는 데이터셋이 (거의) 영어로 되어 있기 때문에 문제없습니다.

In [152]:
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True,
                 remove_punctuation=True, replace_urls=True,
                 replace_numbers=True, stemming=True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for email in X:
            text = email_to_text(email) or ""
            if self.lower_case:
                text = text.lower()
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

이 변환기를 몇 개의 이메일에 적용해 보겠습니다:

In [ ]:
X_few = X_train[:3]
X_few_wordcounts = EmailToWordCounterTransformer().fit_transform(X_few)
X_few_wordcounts

제대로 작동하는 것 같네요!

이제 단어 카운트를 벡터로 변환해야 합니다. 이를 위해서 또 다른 변환기를 만들겠습니다. 이 변환기는 (자주 나타나는 단어 순으로 정렬된) 어휘 목록을 구축하는 `fit()` 메서드와 어휘 목록을 사용해 단어를 벡터로 바꾸는 `transform()` 메서드를 가집니다. 출력은 희소 행렬이 됩니다.

In [154]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.vocabulary_ = {word: index + 1
                            for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)),
                          shape=(len(X), self.vocabulary_size + 1))

In [ ]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors

In [ ]:
X_few_vectors.toarray()

이 행렬은 무엇을 의미하나요? 세 번째 행의 첫 번째 열의 65는 세 번째 이메일이 어휘 목록에 없는 단어를 65개 가지고 있다는 뜻입니다. 그 다음의 0은 어휘 목록에 있는 첫 번째 단어가 한 번도 등장하지 않는다는 뜻이고 그 다음의 1은 한 번 나타난다는 뜻입니다. 이 단어들이 무엇인지 확인하려면 어휘 목록을 보면 됩니다. 첫 번째 단어는 "the"이고 두 번째 단어는 "of"입니다.

In [ ]:
vocab_transformer.vocabulary_

이제 스팸 분류기를 훈련시킬 준비를 마쳤습니다! 전체 데이터셋을 변환시켜보죠:

In [158]:
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
    ("email_to_wordcount", EmailToWordCounterTransformer()),
    ("wordcount_to_vector", WordCounterToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(X_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(max_iter=1000, random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=3)
score.mean()

98.5%가 넘네요. 첫 번째 시도치고 나쁘지 않습니다! :) 그러나 이 데이터셋은 비교적 쉬운 문제입니다. 더 어려운 데이터셋에 적용해 보면 결과가 그리 높지 않을 것입니다. 여러개의 모델을 시도해 보고 제일 좋은 것을 골라 교차 검증으로 세밀하게 튜닝해 보세요.

하지만 전체 내용을 파악했으므로 여기서 멈추겠습니다. 테스트 세트에서 정밀도/재현율을 출력해 보겠습니다:

In [ ]:
from sklearn.metrics import precision_score, recall_score

X_test_transformed = preprocess_pipeline.transform(X_test)

log_clf = LogisticRegression(max_iter=1000, random_state=42)
log_clf.fit(X_train_transformed, y_train)

y_pred = log_clf.predict(X_test_transformed)

print(f"Precision: {precision_score(y_test, y_pred):.2%}")
print(f"Recall: {recall_score(y_test, y_pred):.2%}")